In [8]:
import pandas as pd
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
import numpy as np
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold

#Nome: Imunoterapia
#Esse dataset fala sobre o resultado do tratamento de verrugas utilizando Imunoterapia
#Possui 8 atributos


df = pd.read_excel('Immunotherapy.xlsx')

x = df[['sex','age','Time','Number_of_Warts', 'Type','Area','induration_diameter']]
y = df[['Result_of_Treatment']]

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.25, random_state=0)

#rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=2,random_state=36851234)
#x_treino, x_teste, y_treino, y_teste = rskf.split(x, y)

#Substituição dos valores das colunas:
df.loc[df['Result_of_Treatment'] == 1, 'Result_of_Treatment'] = "Sucesso"
df.loc[df['Result_of_Treatment'] == 0, 'Result_of_Treatment'] = "Fracasso"

#Teste Análise Discriminatória Linear (maior score)
clf = LinearDiscriminantAnalysis()
clf.fit(x_treino, y_treino)

print(clf.score(x_teste, y_teste))

print(pd.crosstab(y_teste.values.ravel(), clf.predict(x_teste), rownames=['Real'], colnames=['Predito T'], margins=True))

#Teste Análise Discriminatória Quadrática
'''clf = QuadraticDiscriminantAnalysis()
clf.fit(x_treino, y_treino.values.ravel())

print(clf.score(x_teste, y_teste))

print(pd.crosstab(y_teste.values.ravel(), clf.predict(x_teste), rownames=['Real'], colnames=['Predito T'], margins=True))

#Teste Floresta Randômica
clf = RandomForestClassifier()
clf.fit(x_treino, y_treino.values.ravel())

print(clf.score(x_teste, y_teste))

print(pd.crosstab(y_teste.values.ravel(), clf.predict(x_teste), rownames=['Real'], colnames=['Predito T'], margins=True))

#Teste Bagging
clf = BaggingClassifier()
clf.fit(x_treino, y_treino.values.ravel())

print(clf.score(x_teste, y_teste))

print(pd.crosstab(y_teste.values.ravel(), clf.predict(x_teste), rownames=['Real'], colnames=['Predito T'], margins=True))'''

0.8260869565217391
Predito T   1  All
Real              
0           4    4
1          19   19
All        23   23


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


"clf = QuadraticDiscriminantAnalysis()\nclf.fit(x_treino, y_treino.values.ravel())\n\nprint(clf.score(x_teste, y_teste))\n\nprint(pd.crosstab(y_teste.values.ravel(), clf.predict(x_teste), rownames=['Real'], colnames=['Predito T'], margins=True))\n\n#Teste Floresta Randômica\nclf = RandomForestClassifier()\nclf.fit(x_treino, y_treino.values.ravel())\n\nprint(clf.score(x_teste, y_teste))\n\nprint(pd.crosstab(y_teste.values.ravel(), clf.predict(x_teste), rownames=['Real'], colnames=['Predito T'], margins=True))\n\n#Teste Bagging\nclf = BaggingClassifier()\nclf.fit(x_treino, y_treino.values.ravel())\n\nprint(clf.score(x_teste, y_teste))\n\nprint(pd.crosstab(y_teste.values.ravel(), clf.predict(x_teste), rownames=['Real'], colnames=['Predito T'], margins=True))"